In [1]:
! conda update matplotlib -yq



# All requested packages already installed.
# packages in environment at /opt/conda:
#
matplotlib                2.1.0                    py36_1    conda-forge


In [2]:
#Load our packages
! conda install geopandas -yq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold





# All requested packages already installed.
# packages in environment at /opt/conda:
#
geopandas                 0.3.0                    py36_0    conda-forge


In [3]:
#Change Directory
import os
os.chdir('Project Files')

#Import state abbreviations dictionary
from us_state_abbrev import *

In [308]:
#Loan action data

loan = pd.read_excel('Loan action.csv')
loan.columns

#Strategy is to merge on county and year, in the format "<state abbreviation>-<county name>"
#Rename the county to CTYNAMES for merging purposes
loan = loan.rename(columns={'county': 'CTYNAME'})
#Convert year to string
loan['year'] = loan['year'].apply(str)
#Convert state name to state abbreviation
loan.state = loan.state.replace(us_state_abbrev, regex=True)
#Replace blank county names with a null string
loan.CTYNAME = loan.CTYNAME.replace(np.nan, '')
#convert to the inteded format
loan.CTYNAME = loan.state + "-" + loan.CTYNAME



In [309]:
## Now we want to convert the actions to columns

#Find unique column names
action = loan.action.unique()
#Get the original column names
b = loan.columns 
#Combine the two
labels = np.append(b,action)

#Remove NaN, action, state, and count from labels
index = np.argwhere(pd.isnull(labels))
labels = np.delete(labels, index)
index = np.argwhere(labels=='action')
labels = np.delete(labels, index)
index = np.argwhere(labels=='state')
labels = np.delete(labels, index)
index = np.argwhere(labels=='count')
labels = np.delete(labels, index)

#Create a template dataframe with proper column names
loan2 = pd.DataFrame(columns = labels)
loan2.columns

Index(['year', 'CTYNAME', 'Application approved but not accepted',
       'Application denied by financial institution', 'Loan originated',
       'Preapproval request approved but not accepted',
       'Preapproval request denied by financial institution'],
      dtype='object')

In [310]:
#populate the county column with unique county names
loan2.CTYNAME = loan.CTYNAME.unique()

loan2

,year,CTYNAME,Application approved but not accepted,Application denied by financial institution,Loan originated,Preapproval request approved but not accepted,Preapproval request denied by financial institution
0,NaN,AL-,NaN,NaN,NaN,NaN,NaN
1,NaN,AL-Autauga County,NaN,NaN,NaN,NaN,NaN
2,NaN,AL-Baldwin County,NaN,NaN,NaN,NaN,NaN
3,NaN,AL-Barbour County,NaN,NaN,NaN,NaN,NaN
4,NaN,AL-Bibb County,NaN,NaN,NaN,NaN,NaN
5,NaN,AL-Blount County,NaN,NaN,NaN,NaN,NaN
6,NaN,AL-Bullock County,NaN,NaN,NaN,NaN,NaN
7,NaN,AL-Butler County,NaN,NaN,NaN,NaN,NaN
8,NaN,AL-Calhoun County,NaN,NaN,NaN,NaN,NaN
9,NaN,AL-Chambers County,NaN,NaN,NaN,NaN,NaN


In [311]:
loanaction = loan.pivot_table(values = 'count', index = ['year', 'CTYNAME'] , columns = 'action' ).reset_index()
loanaction.columns.name = ''
loanaction.fillna(value = 0, inplace=True)

In [319]:
del loanaction2

In [312]:
loanaction

,year,CTYNAME,Application approved but not accepted,Application denied by financial institution,Loan originated,Preapproval request approved but not accepted,Preapproval request denied by financial institution
0,2007,AK-,10.0,49.0,78.0,0.0,0.0
1,2007,AK-Aleutians East Borough,0.0,2.0,4.0,0.0,0.0
2,2007,AK-Aleutians West Census Area,1.0,5.0,7.0,0.0,0.0
3,2007,AK-Anchorage Municipality,429.0,802.0,5405.0,0.0,1.0
4,2007,AK-Bethel Census Area,6.0,20.0,51.0,0.0,0.0
5,2007,AK-Bristol Bay Borough,4.0,2.0,9.0,0.0,0.0
6,2007,AK-Denali Borough,2.0,2.0,20.0,0.0,0.0
7,2007,AK-Dillingham Census Area,4.0,3.0,13.0,0.0,0.0
8,2007,AK-Fairbanks North Star Borough,143.0,256.0,1642.0,0.0,0.0
9,2007,AK-Haines Borough,1.0,3.0,26.0,0.0,0.0


In [7]:
##Don't run this. It's a pivot table function. It is not efficient. 
# I did not know this. I spent too much time on this because I'm a fucking idiot. Sorry for screwing our group.
##Convert the dataframe to the new format 
## Warning: This code takes a long-ass time to run. Try not to do it if you don't have to.

#initialize a blank dataframe that will be our eventual output
#loanaction = pd.DataFrame()
#iterate through all years in datafile
#for x in range(loan.year.min(), loan.year.max()+1):
#    #set temp dataframe to our loan2 template
#    temp = loan2.copy()
#    #set year column to x (current year)
#    temp.year = x
#    #Get the portion of the datafile pertaining to the current year
#    loanyear = loan.loc[loan['year']==x]
#    #iterrate through the loanyear dataframe
#    for index, row in loanyear.iterrows():
#        #check if there is an entry on the action line
#        if pd.isnull(row.action) == False: 
#            #assign the count of the non-null entry to the identified column 
#            temp[row.action].loc[temp['CTYNAME']== row.CTYNAME] = row['count']
#    #append the temp dataframe to the loanaction
#    loanaction = loanaction.append(temp)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
##save as loanaction.csv
loanaction.to_csv('loanaction.csv')
loanaction

,year,CTYNAME,Application approved but not accepted,Application denied by financial institution,Loan originated,Preapproval request approved but not accepted,Preapproval request denied by financial institution
0,2007,AL-,377,907,221,1,NaN
1,2007,AL-Autauga County,156,260,1165,NaN,NaN
2,2007,AL-Baldwin County,539,863,4037,NaN,NaN
3,2007,AL-Barbour County,25,73,154,NaN,NaN
4,2007,AL-Bibb County,58,112,221,NaN,NaN
5,2007,AL-Blount County,109,207,830,1,1
6,2007,AL-Bullock County,5,28,38,NaN,NaN
7,2007,AL-Butler County,29,77,145,NaN,NaN
8,2007,AL-Calhoun County,193,458,1562,1,1
9,2007,AL-Chambers County,35,137,407,NaN,NaN


In [287]:
#2000 census geographic area, note that the header is on line 1
data1 = pd.read_csv('DEC_00_SF1_GCTPH1.US05PR_with_ann.csv', header = 1, encoding='latin-1')
data1 = data1.rename(columns={'Geographical Area': 'CTYNAME'})

In [288]:
data1.columns

Index(['Id', 'Id2', 'Geography', 'Target Geo Id', 'Target Geo Id2', 'CTYNAME',
       'Geographical Area.1', 'Population', 'Housing  units',
       'Area in square miles - Total area',
       'Area in square miles - Water area', 'Area in square miles - Land area',
       'Density per square mile of land area - Population',
       'Density per square mile of land area - Housing  units'],
      dtype='object')

In [289]:
#reformat CTYNAME to our common format
temp=data1.CTYNAME.str.split("-", 2, expand=True)
temp[1] = temp[1].replace(us_state_abbrev, regex=True).str.strip()
temp[2] = temp[2].str.strip()
data1.CTYNAME = temp[1] + "-" + temp[2]
data1.CTYNAME

0                       NaN
1                       NaN
2         AL-Autauga County
3         AL-Baldwin County
4         AL-Barbour County
5            AL-Bibb County
6          AL-Blount County
7         AL-Bullock County
8          AL-Butler County
9         AL-Calhoun County
10       AL-Chambers County
11       AL-Cherokee County
12        AL-Chilton County
13        AL-Choctaw County
14         AL-Clarke County
15           AL-Clay County
16       AL-Cleburne County
17         AL-Coffee County
18        AL-Colbert County
19        AL-Conecuh County
20          AL-Coosa County
21      AL-Covington County
22       AL-Crenshaw County
23        AL-Cullman County
24           AL-Dale County
25         AL-Dallas County
26         AL-DeKalb County
27         AL-Elmore County
28       AL-Escambia County
29         AL-Etowah County
               ...         
3242                    NaN
3243                    NaN
3244                    NaN
3245                    NaN
3246                

In [290]:
#create new dataframe, duplicate it for each subsequent year
area = data1[['CTYNAME', 'Area in square miles - Land area']]
censusarea00 = pd.DataFrame()
for x in range(2007, 2011):
    temp = area.copy()
    temp['year'] = str(x)
    censusarea00 = censusarea00.append(temp)

In [291]:
#Census population estimates read
popnum00 = pd.read_csv('co-est2010-alldata.csv', encoding='latin-1')

In [292]:
#reformat the county name to our preferred format
popnum00.STNAME = popnum00.STNAME.replace(us_state_abbrev, regex=True)
popnum00.CTYNAME = popnum00.STNAME + "-" + popnum00.CTYNAME


In [293]:
#reformat so that each year isn't its own set of variables
basepop00 = popnum00[['CTYNAME','CENSUS2000POP']].copy()
population00 = pd.DataFrame()
for x in range(2007, 2011):
    temporary = basepop00.copy()
    year = str(x)
    for y in popnum00.columns:
        if y.endswith(year):
            label = y.split(year)[0]
            temporary[label] = popnum00[y]
    temporary['year'] = year
    population00 = population00.append(temporary)
    

In [294]:
population00.to_csv('population00.csv')

In [295]:
popdensity00 = population00.merge(censusarea00, how='left', on=['year','CTYNAME'])

In [296]:
#2010 census geographic area, note that the header is on line 1
data = pd.read_csv('DEC_10_SF1_GCTPH1.ST05_with_ann.csv', header = 1, encoding='latin-1')
data = data.rename(columns={'Geographic area': 'CTYNAME'})

In [297]:
data.columns

Index(['Id', 'Id2', 'Geography', 'Target Geo Id', 'Target Geo Id2', 'CTYNAME',
       'Geographic area.1', 'Population', 'Housing units',
       'Area in square miles - Total area',
       'Area in square miles - Water area', 'Area in square miles - Land area',
       'Density per square mile of land area - Population',
       'Density per square mile of land area - Housing units'],
      dtype='object')

In [298]:
#reformat CTYNAME to our common format
temp=data.CTYNAME.str.split("-", 1, expand=True)
temp[0] = temp[0].replace(us_state_abbrev, regex=True).str.strip()
temp[1] = temp[1].str.strip()
data.CTYNAME = temp[0] + "-" + temp[1]
data.CTYNAME

0                                       NaN
1                         AL-Autauga County
2                         AL-Baldwin County
3                         AL-Barbour County
4                            AL-Bibb County
5                          AL-Blount County
6                         AL-Bullock County
7                          AL-Butler County
8                         AL-Calhoun County
9                        AL-Chambers County
10                       AL-Cherokee County
11                        AL-Chilton County
12                        AL-Choctaw County
13                         AL-Clarke County
14                           AL-Clay County
15                       AL-Cleburne County
16                         AL-Coffee County
17                        AL-Colbert County
18                        AL-Conecuh County
19                          AL-Coosa County
20                      AL-Covington County
21                       AL-Crenshaw County
22                        AL-Cul

In [299]:
#create new dataframe, duplicate it for each subsequent year
area = data[['CTYNAME', 'Area in square miles - Land area']]
censusarea = pd.DataFrame()
for x in range(2010, 2018):
    temp = area.copy()
    temp['year'] = str(x)
    censusarea = censusarea.append(temp)


In [300]:
#Census population estimates read
popnum = pd.read_csv('co-est2016-alldata.csv', encoding='latin-1')

In [301]:
#reformat the county name to our preferred format
popnum.STNAME = popnum.STNAME.replace(us_state_abbrev, regex=True)
popnum.CTYNAME = popnum.STNAME + "-" + popnum.CTYNAME


In [302]:
#reformat so that each year isn't its own set of variables
basepop = popnum[['CTYNAME','CENSUS2010POP']].copy()
population = pd.DataFrame()
for x in range(2010, 2017):
    temporary = basepop.copy()
    year = str(x)
    for y in popnum.columns:
        if y.endswith(year):
            label = y.split(year)[0]
            temporary[label] = popnum[y]
    temporary['year'] = year
    population = population.append(temporary)
    
            

In [303]:
#output to population.csv
population.to_csv('population.csv')

In [304]:
#Merge census area with population data, NEED TO CALCULATE DENSITY
popdensity10 = population.merge(censusarea, how='left', on=['year','CTYNAME'])

In [305]:
popdensity = popdensity00.append(popdensity10)
popdensity['density'] = popdensity['POPESTIMATE']/popdensity['Area in square miles - Land area']
popdensity = popdensity.loc[:, ['CTYNAME', 'year', 'density']]

In [306]:
popdensity

,CTYNAME,year,density
0,AL-Alabama,2007,NaN
1,AL-Autauga County,2007,83.619981
2,AL-Baldwin County,2007,108.278886
3,AL-Barbour County,2007,33.603797
4,AL-Bibb County,2007,34.364316
5,AL-Blount County,2007,88.079121
6,AL-Bullock County,2007,17.503720
7,AL-Butler County,2007,26.009500
8,AL-Calhoun County,2007,184.866713
9,AL-Chambers County,2007,58.137549


In [307]:
files = os.listdir()
files_xls = [f for f in files if f[:8] == 'pa_reimb']   
reimb = pd.DataFrame()
for f in files_xls:
    year = f[-8:-4]
    x = pd.read_excel(f, header=[0,1])
    name = list(filter(lambda x: x.startswith('Medicare enrollees'), x.columns.get_level_values(0)))
    x = x.rename(columns={'County name': 'CTYNAME', name[0]: 'Medicare enrollees '})

    for y in x.columns.levels[0]:
        if y.endswith(year + ')'):
            label = y.split('(' + year)[0]
            x.rename(columns = {y : label}, level=0, inplace=True)
    x['year'] = year        
    reimb = reimb.append(x)
    # if reimb.empty:
       # reimb = reimb.append(x)
    #else :
      #  reimb = reimb.merge(x, how='left', on='County name')


reimb = reimb.rename(columns = {'Unnamed: 0_level_1' : '', 'Unnamed: 1_level_1' : ''})
#a = reimb.describe()
#print (a)
reimb['Index'] = reimb.index
reimb = reimb.sort_values([('year',''), ('Index','')])
reimb.drop(labels = 'Index', axis = 1, inplace=True)       
reimb = reimb.loc[:, reimb.columns.get_level_values(0).isin({'CTYNAME', 'Medicare enrollees ', 'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year'})]                         
reimb.to_csv('reimb.csv')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.


In [240]:
reimb.columns.get_level_values(0).isin({'CTYNAME', 'Medicare enrollees ', 'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year'})                        

array([ True, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True,  True,  True], dtype=bool)

In [244]:
reimb

County ID                CTYNAME Medicare enrollees   \
                                                       
1001           AL-Autauga County                 916   
1003           AL-Baldwin County                4322   
1005           AL-Barbour County                 616   
1007              AL-Bibb County                 386   
1009            AL-Blount County                 785   
1011           AL-Bullock County                 262   
1013            AL-Butler County                 671   
1015           AL-Calhoun County                3029   
1017          AL-Chambers County                1128   
1019          AL-Cherokee County                 598   
1021           AL-Chilton County                 690   
1023           AL-Choctaw County                 455   
1025            AL-Clarke County                 750   
1027              AL-Clay County                 512   
1029          AL-Cleburne County                 339   
1031            AL-Coffee County                1234   
1033           AL-Colbert County                1653   
1035           AL-Conecuh County                 265   
1037             AL-Coosa County                 119   
1039         AL-Covington County                1201   
1041          AL-Crenshaw County                 492   
1043           AL-Cullman County                1968   
1045              AL-Dale County                 969   
1047            AL-Dallas County                 957   
1049            AL-DeKalb County                1659   
1051            AL-Elmore County                1444   
1053          AL-Escambia County                1026   
1055            AL-Etowah County                2985   
1057           AL-Fayette County                 376   
1059          AL-Franklin County                 890   
...                          ...                 ...   
55131       WI-Washington County               11955   
55133         WI-Waukesha County               37711   
55135          WI-Waupaca County                4284   
55137         WI-Waushara County                2171   
55139        WI-Winnebago County                9482   
55141             WI-Wood County                5271   
56001           WY-Albany County                2953   
56003         WY-Big Horn County                1975   
56005         WY-Campbell County                2888   
56007           WY-Carbon County                1894   
56009         WY-Converse County                1777   
56011            WY-Crook County                1041   
56013          WY-Fremont County                5393   
56015           WY-Goshen County                2383   
56017      WY-Hot Springs County                1048   
56019          WY-Johnson County                1512   
56021          WY-Laramie County               11748   
56023          WY-Lincoln County                2274   
56025          WY-Natrona County                8911   
56027         WY-Niobrara County                 452   
56029             WY-Park County                5288   
56031           WY-Platte County                1779   
56033         WY-Sheridan County                4599   
56035         WY-Sublette County                1003   
56037       WY-Sweetwater County                3650   
56039            WY-Teton County                2268   
56041            WY-Uinta County                1709   
56043         WY-Washakie County                1465   
56045           WY-Weston County                1092   
99999           US-United States            26097945   

County ID Total Medicare reimbursements per enrollee (Parts A and B)   \
                                             Age, sex & race-adjusted   
1001                                                 8733.96            
1003                                                 6939.79            
1005                                                 6923.82            
1007                                                 7896.10            
1009                             

In [325]:
reimb2 = reimb
reimb2 = reimb2.reset_index()
reimb2.columns

MultiIndex(levels=[['CTYNAME', 'Durable medical equipment reimbursements per enrollee ', 'Home health agency reimbursements per enrollee ', 'Hospice reimbursements per enrollee ', 'Hospital & skilled nursing facility reimbursements per enrollee ', 'Medicare enrollees ', 'Outpatient facility reimbursements per enrollee ', 'Physician reimbursements per enrollee ', 'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year', 'Index', 'index'], ['', 'Age, sex & race-adjusted', 'Price, age, sex & race-adjusted']],
           labels=[[11, 0, 5, 8, 8, 9], [0, 0, 0, 1, 2, 0]],
           names=['County ID', None])

In [190]:
files = os.listdir()
files_csv = [f for f in files if f[:3] == 'ACS']   
div = pd.DataFrame()
for f in files_csv:
    x = pd.read_csv(f, header = 0, encoding='latin-1')
    name = list(filter(lambda x: x.endswith('EST_VC01'), x.columns.values))
    x = x.rename(columns={'GEO.display-label': 'CTYNAME', name[0]: 'Divorce rate'})
    x = x.loc[:, ['CTYNAME','Divorce rate']]
    x = x.drop(x.index[0])
    temp=x.CTYNAME.str.split(", ", expand=True)
    temp[1] = temp[1].replace(us_state_abbrev, regex=True)
    x.CTYNAME = temp[1] + "-" + temp[0]
    year = '20' + f[4:6]
    x['year'] = year
    div = div.append(x)

#a = div.describe()
#print (a)

In [191]:
div

,CTYNAME,Divorce rate,year
1,AL-Baldwin County,8.9,2006
2,AL-Calhoun County,15.0,2006
3,AL-Cullman County,14.0,2006
4,AL-DeKalb County,12.5,2006
5,AL-Elmore County,12.9,2006
6,AL-Etowah County,13.7,2006
7,AL-Houston County,13.5,2006
8,AL-Jefferson County,12.6,2006
9,AL-Lauderdale County,10.7,2006
10,AL-Lee County,8.4,2006


In [313]:
mydata = loanaction.merge(popdensity, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(div, how='left', on=['year', 'CTYNAME'])
mydata.columns = pd.MultiIndex.from_tuples([(c, '') for c in mydata])


In [314]:
mydata

,year,CTYNAME,Application approved but not accepted,Application denied by financial institution,Loan originated,Preapproval request approved but not accepted,Preapproval request denied by financial institution,density,Divorce rate
,,,,,,,,,
0,2007,AK-,10.0,49.0,78.0,0.0,0.0,NaN,NaN
1,2007,AK-Aleutians East Borough,0.0,2.0,4.0,0.0,0.0,0.390805,NaN
2,2007,AK-Aleutians West Census Area,1.0,5.0,7.0,0.0,0.0,1.095122,NaN
3,2007,AK-Anchorage Municipality,429.0,802.0,5405.0,0.0,1.0,162.989848,10.7
4,2007,AK-Bethel Census Area,6.0,20.0,51.0,0.0,0.0,0.420222,NaN
5,2007,AK-Bristol Bay Borough,4.0,2.0,9.0,0.0,0.0,1.978531,NaN
6,2007,AK-Denali Borough,2.0,2.0,20.0,0.0,0.0,0.139769,NaN
7,2007,AK-Dillingham Census Area,4.0,3.0,13.0,0.0,0.0,0.264153,NaN
8,2007,AK-Fairbanks North Star Borough,143.0,256.0,1642.0,0.0,0.0,13.263755,9.7


In [315]:
mydata = mydata.merge(reimb, how='left', on=[('year' , '') , ('CTYNAME' , '')])

In [316]:
mydata.to_csv('mydata3.csv')

In [321]:
mydata.describe()

Application approved but not accepted  \
                                              
count                          23226.000000   
mean                             103.927753   
std                              421.604854   
min                                0.000000   
25%                                7.000000   
50%                               21.000000   
75%                               68.000000   
max                            25402.000000   

      Application denied by financial institution Loan originated  \
                                                                    
count                                23226.000000    23226.000000   
mean                                   250.529665     1095.560622   
std                                    886.334473     3243.019811   
min                                      0.000000        0.000000   
25%                                     22.000000       59.000000   
50%                                     64.000000      198.000000   
75%                                    183.000000      742.000000   
max                                  46519.000000    86270.000000   

      Preapproval request approved but not accepted  \
                                                      
count                                  23226.000000   
mean                                       0.236545   
std                                        4.691051   
min                                        0.000000   
25%                                        0.000000   
50%                                        0.000000   
75%                                        0.000000   
max                                      525.000000   

      Preapproval request denied by financial institution       density  \
                                                                          
count                                       23226.000000    9921.000000   
mean                                            0.922113     238.077344   
std                                             8.401784     846.976566   
min                                             0.000000       0.038129   
25%                                             0.000000      21.137966   
50%                                             0.000000      46.002490   
75%                                             0.000000     120.417595   
max                                           452.000000   18144.314060   

      Medicare enrollees   \
                            
count         9900.000000   
mean          5466.260101   
std          16606.107546   
min              4.000000   
25%            722.750000   
50%           1884.000000   
75%           4566.250000   
max         471496.000000   

      Total Medicare reimbursements per enrollee (Parts A and B)   \
                                         Age, sex & race-adjusted   
count                                        9891.000000            
mean                                         8499.789888            
std                                          1383.451638            
min                                          1810.900000            
25%                                          7654.250000            
50%                                          8416.510000            
75%                                          9239.400000            
max                                         19227.640000            

                                       
      Price, age, sex & race-adjusted  
count                     9891.000000  
mean                      8973.904051  
std                       1602.765184  
min                       1541.360000  
25%                       7920.460000  
50%                       8852.120000  
75%                       9865.550000  
max                      20259.830000

In [328]:
myvars = ['Application approved but not accepted', 'density', 'Medicare enrollees', ('Total Medicare reimbursements per enrollee (Parts A and B)', 'Age, sex & race-adjusted')]
myvars

['Application approved but not accepted',
 'density',
 'Medicare enrollees',
 ('Total Medicare reimbursements per enrollee (Parts A and B)',
  'Age, sex & race-adjusted')]

In [215]:
out = mydata.to_csv()
f = open('myoutput.csv', 'w')
f.write(out)
f.close

<function TextIOWrapper.close>